# Creating nucleus and timestamp collections from a Matlab graph JSON dump

This demonstration uses bogus image and label array files because for simplicity because the
arrays are not important for constructing the lineage from the graph structures.

In order to set up collections using a json dump you need to create the appropriate
time stamp collection before assigning the labels to tracks in the timestamps.  In real usage
this would be a separate step.  Here we fake it using bogus data.

In [ ]:
%%bash

gunzip --keep bogus_labels.tiff.gz
ls

In [ ]:
json_file = "Gata6Nanog1.json"
img_file = "bogus_image.klb"
label_file = "bogus_labels.tiff"

In [ ]:
import json
from mouse_embryo_labeller import tools

json_graph = json.load(open(json_file))
#json_graph

In [ ]:
parse = tools.ParseMatlabJSONDump(json_graph)
#parse.time_stamps

In [ ]:
%mkdir -p json_destination

In [ ]:
# Make an appropriate time stamp collection using the bogus array files
from mouse_embryo_labeller.tools import load_tiff_array
import pyklb

destination = "./json_destination"
helper = tools.FileSystemHelper(destination)

# get the bogus image arrays
stride = 8 # stride for limiting image sizes -- big because who cares?
img = pyklb.readfull(img_file)
labels = load_tiff_array(label_file)
assert img.shape == labels.shape, "bad shapes " + repr((img.shape, labels.shape))
# reduce the image sizes
s_img = img[:, ::stride, ::stride]
s_labels = labels[:, ::stride, ::stride]

# for this demo only, we make timestamps from the JSON data sharing the same bogus images
# this step takes a while because it copies a lot of array data.
for timestamp in parse.time_stamps:
    print ("initializing timestamp", timestamp)
    helper.add_timestamp(timestamp, s_img, s_labels)

In [ ]:
# save the collections
# get the nucleus collection and timestamp collection
tsc = helper.stored_timestamp_collection()
nc = helper.stored_nucleus_collection()

In [ ]:
# get the nucleus collection and timestamp collection
from mouse_embryo_labeller import viz_controller
from mouse_embryo_labeller import tools

folder = "./json_destination"
nc = tools.get_example_nucleus_collection(folder)
tsc = tools.get_example_timestamp_collection(folder, nc)

In [ ]:
json_file = "Gata6Nanog1.json"

import json
from mouse_embryo_labeller import tools

json_graph = json.load(open(json_file))

parse.install_lineage_in_collections(tsc, nc)

tsc.store_all()
nc.save_json()

In [ ]:
%ls json_destination/ts0*

In [ ]:
from mouse_embryo_labeller import viz_controller
v = viz_controller.VizController(folder, tsc, nc)

In [ ]:
v.make_widget()

In [ ]:
parse.node2parent